In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset


import time

In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',

        }
head = pd.DataFrame([head])
head.to_csv("./Free_Adversarial_Training.csv", mode='a', index=False)


In [3]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,

        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./Free_Adversarial_Training.csv", mode='a', index=False, header=False)



In [4]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense_Label/UNSW_Def4/y_pred_{attack_name}{eps}_Def4.npy", all_preds)

        

In [5]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_val.npy')

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [7]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [8]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [10]:
def fgsm(grad, epsilon):
    sign_grad = grad.sign()
    return epsilon * sign_grad
    

In [11]:
# Initialize model, optimizer, and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

# Early stopping variables
min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')

# Adversarial training parameters
epsilon = 0.03  # Step size for FGSM
clip_eps = 0.1  # Maximum perturbation
n_repeats = 4   # Number of iterations to update noise

global_noise_data = torch.zeros(train_loader.batch_size, input_shape).to(device)


In [12]:
# from torch.autograd import Variable


# for epoch in range(10):
#     model.train()
#     train_loss = 0.0
#     for inputs, labels in train_loader:
#         inputs, labels = inputs.to(device), labels.to(device)
#         for j in range(n_repeats):
#             noise_batch = Variable(global_noise_data[0:inputs.size(0)], requires_grad=True).to(device)
#             inputs_adv = inputs + noise_batch
#             inputs_adv.clamp_(0, 1.0)
#             optimizer.zero_grad()
#             outputs = model(inputs_adv)
#             loss = loss_function(outputs, labels)
#             loss.backward()
#             pert = fgsm(noise_batch.grad, epsilon)
#             global_noise_data[0:inputs.size(0)] += pert.data
#             global_noise_data.clamp_(-clip_eps, clip_eps)
#             optimizer.step()
#             train_loss += loss.item()

#     avg_train_loss = train_loss / len(train_loader)

#     model.eval()
#     val_train_loss = 0.0
#     correct_predictions = 0
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             loss = loss_function(outputs, labels)
#             val_train_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             correct_predictions += (predicted == labels).sum().item()

#     avg_val_loss = val_train_loss / len(val_loader)
#     val_accuracy = correct_predictions / len(val_dataset)

#     print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

#     # Early stopping check using min_delta
#     if best_loss - avg_val_loss > min_delta:
#         best_loss = avg_val_loss
#         patience_counter = 0
#     else:
#         patience_counter += 1

#     if patience_counter >= patience:
#         print("Early stopping triggered")
#         break

In [13]:

model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/Free_Adversarial_Training/Free_Adversarial_Training.pt"))

/tmp/ipykernel_179/1052584200.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/Fre

<All keys matched successfully>

In [14]:
# calculate_performance_metrics(x_test, y_test, model, 'DNN', 'baseline', '0')

In [15]:
def print_empty_file():
    new_row = {
        "model" : "0",
        "attack_model" : "0",
        'epsilon': "0",
        'Accuracy': "0",
        'Macro Precision': "0",
        'Weighted Precision': "0",
        'Macro Recall': "0",
        'Weighted Recall': "0",
        'Macro F1': "0",
        'Weighted F1': "0",
    }
    new_row_df = pd.DataFrame([new_row])
    new_row_df.to_csv("./Free_Adversarial_Training.csv", mode='a', index=False, header=False)

In [14]:
epsilon = 0
Def = "Def4"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
(856, 56) (856,)
Accuracy: 0.6869158878504673

macro
Precision: 0.42488038277511964
Recall: 0.49178591048211995
F1 Score: 0.4177639259717978

weighted
Precision: 0.5366308187631356
Recall: 0.6869158878504673
F1 Score: 0.5787001506002806

start XGB 100 BIM
(1223, 56) (1223,)
Accuracy: 0.9133278822567457

macro
Precision: 0.5209106688664329
Recall: 0.6674580236718965
F1 Score: 0.5203566671599822

weighted
Precision: 0.9844655839944969
Recall: 0.9133278822567457
F1 Score: 0.9459867366279311

start XGB 100 FGSM
(1225, 56) (1225,)
Accuracy: 0.9355102040816327

macro
Precision: 0.4965337954939341
Recall: 0.47082990961380444
F1 Score: 0.48334036271615355

weighted
Precision: 0.9865822516181516
Recall: 0.9355102040816327
F1 Score: 0.960367708449892

start XGB 100 PGD
(1223, 56) (1223,)
Accuracy: 0.9133278822567457

macro
Precision: 0.5209106688664329
Recall: 0.6674580236718965
F1 Score: 0.5203566671599822

weighted
Precision: 0.9844655839944969
Recall: 0.9133278822567457

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9480840543881335

macro
Precision: 0.5
Recall: 0.47404202719406674
F1 Score: 0.4866751269035533

weighted
Precision: 1.0
Recall: 0.9480840543881335
F1 Score: 0.9733502538071066

start RF 50 FGSM
(778, 56) (778,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9832904884318766

macro
Precision: 0.5
Recall: 0.4916452442159383
F1 Score: 0.4957874270900843

weighted
Precision: 1.0
Recall: 0.9832904884318766
F1 Score: 0.9915748541801684

start RF 50 PGD
(809, 56) (809,)


...

Accuracy: 0.9713024282560706

macro
Precision: 0.5
Recall: 0.4856512141280353
F1 Score: 0.49272116461366183

weighted
Precision: 1.0
Recall: 0.9713024282560706
F1 Score: 0.9854423292273236

start RF 20 VNIFGSM
(443, 56) (443,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9841986455981941

macro
Precision: 0.5
Recall: 0.49209932279909707
F1 Score: 0.4960182025028441

weighted
Precision: 1.0
Recall: 0.9841986455981941
F1 Score: 0.9920364050056883

start RF 1 baseline
(670, 56) (670,)
Accuracy: 0.5791044776119403

macro
Precision: 0.6536195286195285
Recall: 0.5619196428571429
F1 Score: 0.4945749871597329

weighted
Precision: 0.6493039851248805
Recall: 0.5791044776119403
F1 Score: 0.5038300408588797

start RF 1 BIM
(39, 56) (39,)
Accuracy: 0.8461538461538461

macro
Precision: 0.5
Recall: 0.4230769230769231
F1 Score: 0.4583333333333333

weighted
Precision: 1.0
Recall: 0.8461538461538461
F1 Score: 0.9166666666666666

start RF 1 FGSM
(14, 56) (14,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 1 PGD
(39, 56) (39,)
Accuracy: 0.8461538461538461

macro
Precision: 0.5
Recall: 0.4230769230769231
F1 Score: 0.4583333333333333

weighted
Precision: 1.0
Recall: 0.8461538461538461
F

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

Accuracy: 0.8483935742971888

macro
Precision: 0.45187165775401067
Recall: 0.4663355408388521
F1 Score: 0.45898967952199893

weighted
Precision: 0.8220797629018748
Recall: 0.8483935742971888
F1 Score: 0.8350294169617088

start RF 1 ZOO
(1699, 56) (1699,)
Accuracy: 0.6097704532077692

macro
Precision: 0.6690176750509857
Recall: 0.5791275471318855
F1 Score: 0.5316202106891369

weighted
Precision: 0.6621062935809404
Recall: 0.6097704532077692
F1 Score: 0.5479484313882748

start RF 1 CaFA
(536, 56) (536,)
Accuracy: 0.4216417910447761

macro
Precision: 0.4608353339397283
Recall: 0.45912380359505095
F1 Score: 0.42098880727039567

weighted
Precision: 0.5102532793290159
Recall: 0.4216417910447761
F1 Score: 0.42730098375607295

start RF 1 SINIFGSM
(7, 56) (7,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 1 VNIFGSM
(13, 56) (13,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Rec

In [13]:
epsilon = 0
Def = "Def4"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ExcludeCaFA"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
(818, 56) (818,)
Accuracy: 0.7298288508557457

macro
Precision: 0.5116382373845061
Recall: 0.5010104516178796
F1 Score: 0.43893135693956914

weighted
Precision: 0.6188002095703807
Recall: 0.7298288508557457
F1 Score: 0.6305582289930097

start XGB 100 BIM
(1282, 56) (1282,)
Accuracy: 0.9227769110764431

macro
Precision: 0.6341190966204506
Recall: 0.8893594858591616
F1 Score: 0.6864325332608615

weighted
Precision: 0.9717305755163482
Recall: 0.9227769110764431
F1 Score: 0.9412512694501677

start XGB 100 FGSM
(1506, 56) (1506,)
Accuracy: 0.9415670650730412

macro
Precision: 0.5620391297399722
Recall: 0.7885534279545534
F1 Score: 0.5919704433497537

weighted
Precision: 0.9841201972164458
Recall: 0.9415670650730412
F1 Score: 0.9601252788517524

start XGB 100 PGD
(1282, 56) (1282,)
Accuracy: 0.9227769110764431

macro
Precision: 0.6341190966204506
Recall: 0.8893594858591616
F1 Score: 0.6864325332608615

weighted
Precision: 0.9717305755163482
Recall: 0.9227769110764431
F

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.978494623655914

macro
Precision: 0.5
Recall: 0.489247311827957
F1 Score: 0.4945652173913043

weighted
Precision: 1.0
Recall: 0.978494623655914
F1 Score: 0.9891304347826085

start RF 50 baseline
(1064, 56) (1064,)
Accuracy: 0.8458646616541353

macro
Precision: 0.7422084088750756
Recall: 0.518728429428768
F1 Score: 0.49727456064534714

weighted
Precision: 0.8148274727222096
Recall: 0.8458646616541353
F1 Score: 0.7844876235532652

start RF 50 BIM
(790, 56) (790,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9696202531645569

macro
Precision: 0.5
Recall: 0.48481012658227846
F1 Score: 0.4922879177377892

weighted
Precision: 1.0
Recall: 0.9696202531645569
F1 Score: 0.9845758354755784

start RF 50 FGSM
(801, 56) (801,)


...

Accuracy: 0.959349593495935

macro
Precision: 0.5
Recall: 0.4796747967479675
F1 Score: 0.4896265560165975

weighted
Precision: 1.0
Recall: 0.959349593495935
F1 Score: 0.979253112033195

start RF 1 AutoPGD
(275, 56) (275,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9709090909090909

macro
Precision: 0.5
Recall: 0.48545454545454547
F1 Score: 0.492619926199262

weighted
Precision: 1.0
Recall: 0.9709090909090909
F1 Score: 0.985239852398524

start RF 1 ZOO
(363, 56) (363,)
Accuracy: 0.7382920110192838

macro
Precision: 0.5864767255216693
Recall: 0.5085225025705924
F1 Score: 0.45370297029702966

weighted
Precision: 0.6626018686794989
Recall: 0.7382920110192838
F1 Score: 0.6437910700160925

start RF 1 CaFA
(15, 56) (15,)
Accuracy: 0.2

macro
Precision: 0.5
Recall: 0.1
F1 Score: 0.16666666666666666

weighted
Precision: 1.0
Recall: 0.2
F1 Score: 0.3333333333333333

start RF 1 SINIFGSM
(2, 56) (2,)
Accuracy: 0.5

macro
Precision: 0.5
Recall: 0.25
F1 Score: 0.3333333333333333

weighted
Precision: 1.0
Recall: 0.5
F1 Score: 0.6666666666666666

start RF 1 VNIFGSM
(4, 56) (4,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start DT 100 baseline
(489, 56) (489,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

Accuracy: 0.9550102249488752

macro
Precision: 0.9163583509513742
Recall: 0.7036845507433742
F1 Score: 0.7681465517241379

weighted
Precision: 0.951965438375768
Recall: 0.9550102249488752
F1 Score: 0.9473483886890911

start DT 100 BIM
(6287, 56) (6287,)
Accuracy: 0.7863846031493558

macro
Precision: 0.5650581203555802
Recall: 0.5596411191003904
F1 Score: 0.5619796964874453

weighted
Precision: 0.7774216756946112
Recall: 0.7863846031493558
F1 Score: 0.7817469017450097

start DT 100 FGSM
(6341, 56) (6341,)
Accuracy: 0.794038795142722

macro
Precision: 0.524432928312076
Recall: 0.5175902038516491
F1 Score: 0.5179064005569953

weighted
Precision: 0.7633114611356921
Recall: 0.794038795142722
F1 Score: 0.77735231995417

start DT 100 PGD
(6287, 56) (6287,)
Accuracy: 0.7863846031493558

macro
Precision: 0.5650581203555802
Recall: 0.5596411191003904
F1 Score: 0.5619796964874453

weighted
Precision: 0.7774216756946112
Recall: 0.7863846031493558
F1 Score: 0.7817469017450097

start DT 100 DF
(851,

In [13]:
epsilon = 0
Def = "Def4"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

total_num_attack = ["2", "3"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    print(x_test_adv.shape, y_test_adv.shape)
    
                    m_per_name = f"{m_name}{p}_Exclude{num_attack}Attack"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()


start XGB 100 baseline
(2061, 56) (2061,)
Accuracy: 0.6783114992721979

macro
Precision: 0.49657612417256336
Recall: 0.4997051191318707
F1 Score: 0.42096069868995634

weighted
Precision: 0.5666858377132695
Recall: 0.6783114992721979
F1 Score: 0.5642455330752656

start XGB 100 BIM
(740, 56) (740,)
Accuracy: 0.7945945945945946

macro
Precision: 0.5212927922049676
Recall: 0.8343579234972678
F1 Score: 0.48433001412040855

weighted
Precision: 0.9879685113797081
Recall: 0.7945945945945946
F1 Score: 0.8756741887951659

start XGB 100 FGSM
(906, 56) (906,)
Accuracy: 0.8774834437086093

macro
Precision: 0.5131578947368421
Recall: 0.9385382059800664
F1 Score: 0.49289766280916725

weighted
Precision: 0.996775880097595
Recall: 0.8774834437086093
F1 Score: 0.9315886676374281

start XGB 100 PGD
(740, 56) (740,)
Accuracy: 0.7945945945945946

macro
Precision: 0.5212927922049676
Recall: 0.8343579234972678
F1 Score: 0.48433001412040855

weighted
Precision: 0.9879685113797081
Recall: 0.7945945945945946
F1

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8144329896907216

macro
Precision: 0.5
Recall: 0.4072164948453608
F1 Score: 0.44886363636363635

weighted
Precision: 1.0
Recall: 0.8144329896907216
F1 Score: 0.8977272727272727

start RF 100 FGSM
(289, 56) (289,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9446366782006921

macro
Precision: 0.5
Recall: 0.47231833910034604
F1 Score: 0.48576512455516013

weighted
Precision: 1.0
Recall: 0.9446366782006921
F1 Score: 0.9715302491103204

start RF 100 PGD
(388, 56) (388,)


...

Accuracy: 0.06299212598425197

macro
Precision: 0.5
Recall: 0.031496062992125984
F1 Score: 0.05925925925925926

weighted
Precision: 1.0
Recall: 0.06299212598425197
F1 Score: 0.11851851851851852

start RF 1 FGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input1/x_test_adv_FGSM_Def4.npy not found
start RF 1 PGD
(127, 56) (127,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.06299212598425197

macro
Precision: 0.5
Recall: 0.031496062992125984
F1 Score: 0.05925925925925926

weighted
Precision: 1.0
Recall: 0.06299212598425197
F1 Score: 0.11851851851851852

start RF 1 DF
(18, 56) (18,)
Accuracy: 0.6666666666666666

macro
Precision: 0.675
Recall: 0.675
F1 Score: 0.6666666666666666

weighted
Precision: 0.6833333333333333
Recall: 0.6666666666666666
F1 Score: 0.6666666666666666

start RF 1 AutoPGD
(835, 56) (835,)
Accuracy: 0.7029940119760479

macro
Precision: 0.5053903398319233
Recall: 0.5264914476428869
F1 Score: 0.45541950685853744

weighted
Precision: 0.9207391052478106
Recall: 0.7029940119760479
F1 Score: 0.7909423264050037

start RF 1 ZOO
(1109, 56) (1109,)
Accuracy: 0.5942290351668169

macro
Precision: 0.4810364477173603
Recall: 0.48735929407811596
F1 Score: 0.46994833841735417

weighted
Precision: 0.5397941274824956
Recall: 0.5942290351668169
F1 Score: 0.5548850529556276

start RF 1 CaFA
(58, 56) (58,)
Accuracy: 0.22413793103448276

macro
Prec

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classifi

Accuracy: 0.8725905030559473

macro
Precision: 0.8253855838841675
Recall: 0.5121409109059732
F1 Score: 0.4904218217501426

weighted
Precision: 0.8606381906625237
Recall: 0.8725905030559473
F1 Score: 0.8171947170034773

start DT 100 BIM
(4468, 56) (4468,)
Accuracy: 0.8115487914055506

macro
Precision: 0.7094920986624351
Recall: 0.8333682006396737
F1 Score: 0.7345840199099942

weighted
Precision: 0.8874221821336136
Recall: 0.8115487914055506
F1 Score: 0.8324054127750863

start DT 100 FGSM
(4259, 56) (4259,)
Accuracy: 0.8391641230335759

macro
Precision: 0.7404112182630489
Recall: 0.8467235174402615
F1 Score: 0.7696198225432644

weighted
Precision: 0.8902670248029395
Recall: 0.8391641230335759
F1 Score: 0.8534593403565846

start DT 100 PGD
(4468, 56) (4468,)
Accuracy: 0.8115487914055506

macro
Precision: 0.7094920986624351
Recall: 0.8333682006396737
F1 Score: 0.7345840199099942

weighted
Precision: 0.8874221821336136
Recall: 0.8115487914055506
F1 Score: 0.8324054127750863

start DT 100 DF

In [16]:
epsilon = 0

Def = "Def4"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

rec_list = ["Euclidean", "cosine", "manhattan"]
for rec in rec_list:
    for attack in attack_names:
        print(f"start {attack}")
        
        x_path = f"{base_path}/Recommendation_{rec}/x_test_adv_{attack}_{Def}.npy"
        y_path = f"{base_path}/Recommendation_{rec}/y_test_adv_{attack}_{Def}.npy"
    
        try:
            x_test_adv = np.load(x_path)
            y_test_adv = np.load(y_path)
            m_per_name = f"Recommendation_{rec}"
            calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
        except FileNotFoundError:
            print(x_path, "not found")
            print_empty_file()



start baseline
Accuracy: 0.9138321995464853

macro
Precision: 0.8151260504201681
Recall: 0.5400041562759768
F1 Score: 0.5519357828690941

weighted
Precision: 0.8976733550563083
Recall: 0.9138321995464853
F1 Score: 0.8815689361589297

start BIM
Accuracy: 0.6554959785522788

macro
Precision: 0.5327906150017775
Recall: 0.5996347589639599
F1 Score: 0.4908733656042393

weighted
Precision: 0.8748318158988
Recall: 0.6554959785522788
F1 Score: 0.7331891589204955

start FGSM
Accuracy: 0.7620614035087719

macro
Precision: 0.5517007750971163
Recall: 0.6456794972505892
F1 Score: 0.5444449417683446

weighted
Precision: 0.8992531176902864
Recall: 0.7620614035087719
F1 Score: 0.8155736481990409

start PGD
Accuracy: 0.6554959785522788

macro
Precision: 0.5327906150017775
Recall: 0.5996347589639599
F1 Score: 0.4908733656042393

weighted
Precision: 0.8748318158988
Recall: 0.6554959785522788
F1 Score: 0.7331891589204955

start DF
Accuracy: 0.6775818639798489

macro
Precision: 0.5630131362889984
Recall: 0

In [15]:

# torch.save(model.state_dict(), "./Free_Adversarial_Training.pt")

In [17]:
# import time

# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# start_time = time.time()

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

# end_time = time.time()
# result = end_time - start_time
# print(f"Execution Time: {result:.6f} seconds")